# CAR-19 Data Analysis

This notebook runs the code in the nircam_calib/comissioning/NRC_19_subarrays module in order to analyze data from CAR-19 (Subarray Verification)

## Table of Contents

* [Goal 1: Register and combine data in pipeline](#goal_1)
* [Goal 2: Confirm positions of subarrays on detectors](#goal_2)
    * [Manual examination of images](#manual_examination)
    * [Compare source locations](#source_locations)
* [Goal 3: Confirm telescope pointing places target in correct location](#goal_3)
    * [Compare source locations to 2MASS catalog](#twomass_comp)
    * [Compare target location to expected location](#targ_star_comp)
    * [Check that target RA, Dec in file matches position in APT](#targ_ra_dec_comp)
* [Goal 4: Confirm same charge accumulation rate in subarrays vs full frame](#goal_4)
    * [Compare photometry in subarrays vs full frame](#photometry_comparison)
    * [Compare level 3 source catalogs](#compare_level3_catalog)
    * [Examine ratio images with the same pointing](#ratios)
* [Goal 5: Identify and characterize latency effects](#goal_5)

In [ ]:
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from glob import glob
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.associations.lib.rules_level2b import Asn_Lv2ImageTSO, Asn_Lv2SpecTSO
from jwst.associations.lib.rules_level3 import Asn_TSO
from jwst.pipeline.calwebb_detector1 import Detector1Pipeline
from jwst.pipeline.calwebb_image2 import Image2Pipeline
from jwst.pipeline.calwebb_spec2 import Spec2Pipeline
from jwst.pipeline.calwebb_image3 import Image3Pipeline
from jwst.pipeline.calwebb_tso3 import Tso3Pipeline

In [ ]:
from nircam_calib.commissioning.NRC_19_subarrays import confirm_subarray_location_via_sources as locations
from nircam_calib.commissioning.NRC_19_subarrays import confirm_subarray_location_via_examination as examination
from nircam_calib.commissioning.NRC_19_subarrays import confirm_telescope_pointing as pointing
from nircam_calib.commissioning.NRC_19_subarrays import confirm_count_rates as count_rates
from nircam_calib.commissioning.NRC_19_subarrays import latency_investigation as latency

In [ ]:
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

In [ ]:
base_dir = '/ifs/jwst/wit/nircam/simulationWG/Imaging/CAR-19/2020_Oct'
#base_dir = 'path_to_simulationWG_CAR-19_directory'

In [ ]:
def ensure_dir_exists(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [ ]:
output_products_dir = os.path.join(base_dir, 'Analysis_products')
ensure_dir_exists(output_products_dir)

In [ ]:
import jwst
print(jwst.__version__)

#### RA and Dec of the target, as defined in the APT file

In goal #2, these will be compared to the TARG_RA, TARG_DEC keyword values

In [ ]:
extended_target_ra, extended_target_dec = '05 21 57.0000', '-69 29 51.00'
ptsrc_target_ra, ptsrc_target_dec = '05 21 55.8098', '-69 29 38.25'

In [ ]:
def ra_dec_dec_degrees(str_value, ra_hours=False):
    """Convert string representation of RA or Dec to decimal degrees
    
    Parameters
    ----------
    str_value : str
        String value of RA or Dec separated by spaces (e.g. '05 21 55.8098')
        since this is the way it is presented in APT
        
    ra_hours : bool
        If True, the input is assumed to be RA in units of hours/minutes/seconds.
        If False, the input is assumed to be degrees/arcmin/arcsec
        
    Returns
    -------
    deg_value : float
        ```str_value``` converted to decimal degrees
    """
    d, m, s = str_value.split(' ')
    d_dec = np.float(d)
    m_dec = np.float(m)
    s_dec = np.float(s)
    if d_dec < 0:
        m_dec = 0. - m_dec
        s_dec = 0. - s_dec
    deg_value = d_dec + (m_dec + (s_dec / 60.)) / 60.
    if ra_hours:
        deg_value *= 15.
    return deg_value

In [ ]:
extended_ra = ra_dec_dec_degrees(extended_target_ra, ra_hours=True)
extended_dec = ra_dec_dec_degrees(extended_target_dec)
point_ra = ra_dec_dec_degrees(ptsrc_target_ra, ra_hours=True)
point_dec = ra_dec_dec_degrees(ptsrc_target_dec)

In [ ]:
print(extended_ra, extended_dec)
print(point_ra, point_dec)

<a id='goal_1'></a>
## Goal 1: Check that calibration pipeline correctly registers and combines subarray data

 Run pipeline through level3 separately for each aperture
 
 examine output:  
      measure FWHM of sources  
      visual inspection of mosaic size/shape

### Calwebb_detector1

In [ ]:
pipeline1_output_dir = os.path.join(base_dir, 'Pipeline_level1/')
ensure_dir_exists(pipeline1_output_dir)

In [ ]:
def run_calwebb_detector1(filename, ta=False):
    m = Detector1Pipeline() #config_file=os.path.join(base_dir, 'pipeline_config_files/calwebb_detector1.cfg'))

    # make changes to the parameters/reference files used
    m.refpix.odd_even_rows = False

    # jump step is way too sensitive
    m.jump.rejection_threshold = 91

    # skip steps you don't want to run
    m.group_scale.skip = True
    m.ipc.skip = True
    m.persistence.skip = True
    
    # skip dark subtraction on TA files
    if ta:
        m.dark_current.skip = True

    # name your output file
    m.save_results = True
    m.output_dir = pipeline1_output_dir
    m.output_file = os.path.basename(filename.replace('_uncal', '_rate'))

    # run the pipeline with these paramters
    m.run(filename)
    print('')
    print("Done running CALDETECTOR1 on {}".format(filename))
    print("Output saved to {}".format(os.path.join(m.output_dir, m.output_file)))
    print('')

#### Run calwebb_detector1

TA images can be used in a few of the tests below, so let's run them as well. We will have to explicitly skip dark current subtraction for them since there are no dark reference files in CRDS for those apertures.

In [ ]:
uncal_files = sorted(glob(os.path.join(base_dir, 'Mirage_Output/j*uncal.fits')))

In [ ]:
ta_uncal_files = ['Mirage_Output/jw01068005001_01101_00001_nrcb5_uncal.fits',
                  'Mirage_Output/jw01068006001_01101_00001_nrcb5_uncal.fits',
                  'Mirage_Output/jw01068007001_01101_00001_nrca5_uncal.fits']
ta_uncal_files = [os.path.join(base_dir, f) for f in ta_uncal_files]

In [ ]:
# Process TA files separately
for f in ta_uncal_files:
    uncal_files.remove(f)

In [ ]:
# Run non-TA files through calwebb_detector1
for filename in uncal_files:
    #for filename in [os.path.join(base_dir, 'Mirage_Output/jw01068007001_01101_00001-seg001_nrca3_uncal.fits')]:
    run_calwebb_detector1(filename)

In [ ]:
# Run TA files through calwebb_detector1.
# This is done separately from the other files so that we can skip dark subtraction
for filename in ta_uncal_files:
    run_calwebb_detector1(filename, ta=True)

### Calwebb_image2

In [ ]:
pipeline2_output_dir = os.path.join(base_dir, 'Pipeline_level2/')
ensure_dir_exists(pipeline2_output_dir)

In [ ]:
def make_level2_association(file_list, asn_filename, rule=None):
    idx = file_list[0].find('nrca1')
    prod_name = file_list[0][0: idx+5]
    if rule is None:
        asn = asn_from_list(file_list, rule=DMSLevel2bBase, product_name=prod_name)
    elif rule.lower() == 'asn_lv2imagetso':
        asn = asn_from_list(file_list, product_name=prod_name, rule=Asn_Lv2ImageTSO)
    elif rule.lower() == 'asn_lv2spectso':
        asn = asn_from_list(file_list, product_name=prod_name, rule=Asn_Lv2SpecTSO)
    outfile = os.path.join(pipeline2_output_dir, asn_filename)
    with open(outfile, 'w') as fh:
        fh.write(asn.dump()[1])

In [ ]:
def run_calwebb_image2(filename):
    result2 = Image2Pipeline()
    result2.save_results = True
    result2.output_dir = pipeline2_output_dir
    result2.run(filename)

In [ ]:
def run_calwebb_spec2(filename):
    result2 = Spec2Pipeline()
    result2.save_results = True
    result2.output_dir = pipeline2_output_dir
    result2.run(filename)

#### 1. Create association files

Create association files. For level2 it's not as important, but let's make one association file for each subarray size

In [ ]:
sub160_asn = os.path.join(pipeline2_output_dir, 'level2_sub160_files_asn.json')
sub160_rate_files = sorted(glob(os.path.join(pipeline1_output_dir, 'jw01068001001*rate.fits')))
make_level2_association(sub160_rate_files, sub160_asn)

sub320_asn = os.path.join(pipeline2_output_dir, 'level2_sub320_files_asn.json')
sub320_rate_files = sorted(glob(os.path.join(pipeline1_output_dir, 'jw01068002001*rate.fits')))
make_level2_association(sub320_rate_files, sub320_asn)

sub640_asn = os.path.join(pipeline2_output_dir, 'level2_sub640_files_asn.json')
sub640_rate_files = sorted(glob(os.path.join(pipeline1_output_dir, 'jw01068003001*rate.fits')))
make_level2_association(sub640_rate_files, sub640_asn)

full_asn = os.path.join(pipeline2_output_dir, 'level2_full_files_asn.json')
full_rate_files = sorted(glob(os.path.join(pipeline1_output_dir, 'jw01068004001*rate.fits')))
make_level2_association(full_rate_files, full_asn)

sub400p_asn = os.path.join(pipeline2_output_dir, 'level2_sub400p_files_asn.json')
sub400p_rate_files = sorted(glob(os.path.join(pipeline1_output_dir, 'jw01068005001*rateints.fits')))
make_level2_association(sub400p_rate_files, sub400p_asn, rule='Asn_Lv2ImageTSO')

sub64p_asn = os.path.join(pipeline2_output_dir, 'level2_sub64p_files_asn.json')
sub64p_rate_files = sorted(glob(os.path.join(pipeline1_output_dir, 'jw01068006001*rateints.fits')))
make_level2_association(sub64p_rate_files, sub64p_asn, rule='Asn_Lv2ImageTSO')

substripe256_asn = os.path.join(pipeline2_output_dir, 'level2_substrip256_files_asn.json')
substripe256_rate_files = sorted(glob(os.path.join(pipeline1_output_dir,
                                                   'jw01068007001*nrca[1234]*rateints.fits')))
make_level2_association(substripe256_rate_files, substripe256_asn, rule='Asn_Lv2ImageTSO')

substripe256_grism_asn = os.path.join(pipeline2_output_dir, 'level2_substrip256_grism_files_asn.json')
substripe256_grism_rate_files = sorted(glob(os.path.join(pipeline1_output_dir,
                                                         'jw01068007001*00002*nrca5*rateints.fits')))
make_level2_association(substripe256_grism_rate_files, substripe256_grism_asn, rule='Asn_Lv2SpecTSO')

#### 2. Run calwebb_image2

Run on imaging mode files

In [ ]:
association_files = [sub160_asn, sub320_asn, sub640_asn, full_asn, sub400p_asn, sub64p_asn, substripe256_asn]
for asn in association_files:
    run_calwebb_image2(asn)

Run calwebb_spec2 on the grism TSO data

In [ ]:
run_calwebb_spec2(substripe256_grism_asn)

### Calwebb_image3

In [ ]:
pipeline3_output_dir = os.path.join(base_dir, 'Pipeline_level3/')
ensure_dir_exists(pipeline3_output_dir)

In [ ]:
def run_calwebb_image3(filename):
    result = Image3Pipeline()
    result.save_results = True
    result.source_catalog.save_results = True
    result.source_catalog.output_dir = pipeline3_output_dir
    result.output_dir = pipeline3_output_dir
    result.run(filename)

In [ ]:
def run_calwebb_tso3(filename):
    result = Tso3Pipeline()
    result.save_results = True
    result.tso_photometry.save_catalog = True
    result.tso_photometry.output_dir = pipeline3_output_dir
    result.output_dir = pipeline3_output_dir
    result.run(filename)

Create association files for various combinations of images. Let's try:

1. Combining all data (FULL, SUB640, SUB320, SUB160, SUB400P, SUB64P)
2. Combine each subarray separately

#### 1. Create association files 

In [ ]:
def make_level3_association(file_list, asn_filename, product_name, rule=None):
    if rule is None:
        asn = asn_from_list(file_list, product_name=product_name)
    elif rule.lower() == 'asn_tso':
        asn = asn_from_list(file_list, product_name=product_name, rule=Asn_TSO)
    outfile = os.path.join(pipeline3_output_dir, asn_filename)
    with open(outfile, 'w') as fh:
        fh.write(asn.dump()[1])

In [ ]:
# Create output names and get file lists
sub160_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub160_sw_files_asn.json')
sub160_sw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068001001*nrcb[1234]_cal.fits')))

sub160_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub160_lw_files_asn.json')
sub160_lw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068001001*nrcb5_cal.fits')))

sub320_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub320_sw_files_asn.json')
sub320_sw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068002001*nrcb[1234]_cal.fits')))

sub320_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub320_lw_files_asn.json')
sub320_lw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068002001*nrcb5_cal.fits')))

sub640_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub640_sw_files_asn.json')
sub640_sw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068003001*nrcb[1234]_cal.fits')))

sub640_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub640_lw_files_asn.json')
sub640_lw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068003001*nrcb5_cal.fits')))

full_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_full_sw_files_asn.json')
full_sw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068004001*nrcb[1234]_cal.fits')))

full_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_full_lw_files_asn.json')
full_lw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068004001*nrcb5_cal.fits')))

sub400p_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub400p_sw_files_asn.json')
sub400p_sw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068005001*nrcb[1234]_calints.fits')))

sub400p_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub400p_lw_files_asn.json')
sub400p_lw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068005001*00002_nrcb5_calints.fits')))

# Omit the TA image
sub64p_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub64p_sw_files_asn.json')
sub64p_sw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068006001*nrcb[1234]_calints.fits')))

# Omit the TA image
sub64p_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_sub64p_lw_files_asn.json')
sub64p_lw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068006001*00002_nrcb5_calints.fits')))

substripe256_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_substripe256_sw_files_asn.json')
substripe256_sw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068007001*nrc?[1234]_calints.fits')))

substripe256_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_substripe256_lw_files_asn.json')
substripe256_lw_cal_files = sorted(glob(os.path.join(pipeline2_output_dir, 'jw01068007001*00002*nrca5_calints.fits')))

all_sw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_all_sw_subarrays_asn.json')
all_sw_cal_files = sub160_sw_cal_files + sub320_sw_cal_files + sub640_sw_cal_files + full_sw_cal_files

all_lw_asn_3 = os.path.join(pipeline3_output_dir, 'level3_all_lw_subarrays_asn.json')
all_lw_cal_files = sub160_lw_cal_files + sub320_lw_cal_files + sub640_lw_cal_files + full_lw_cal_files

In [ ]:
# Create Image3 association files
make_level3_association(sub160_sw_cal_files, sub160_sw_asn_3, 'sub160_sw')
make_level3_association(sub160_lw_cal_files, sub160_lw_asn_3, 'sub160_lw')

make_level3_association(sub320_sw_cal_files, sub320_sw_asn_3, 'sub320_sw')
make_level3_association(sub320_lw_cal_files, sub320_lw_asn_3, 'sub320_lw')

make_level3_association(sub640_sw_cal_files, sub640_sw_asn_3, 'sub640_sw')
make_level3_association(sub640_lw_cal_files, sub640_lw_asn_3, 'sub640_lw')

make_level3_association(full_sw_cal_files, full_sw_asn_3, 'full_sw')
make_level3_association(full_lw_cal_files, full_lw_asn_3, 'full_lw')

make_level3_association(sub400p_sw_cal_files, sub400p_sw_asn_3, 'sub400p_sw', rule='asn_tso')
make_level3_association(sub400p_lw_cal_files, sub400p_lw_asn_3, 'sub400p_lw', rule='asn_tso')

make_level3_association(sub64p_sw_cal_files, sub64p_sw_asn_3, 'sub64p_sw', rule='asn_tso')
make_level3_association(sub64p_lw_cal_files, sub64p_lw_asn_3, 'sub64p_lw', rule='asn_tso')

make_level3_association(substripe256_sw_cal_files, substripe256_sw_asn_3, 'substrip256_sw', rule='asn_tso')
make_level3_association(substripe256_lw_cal_files, substripe256_lw_asn_3, 'substripe256_lw', rule='asn_tso')

make_level3_association(all_sw_cal_files, all_sw_asn_3, 'all_subarrays_sw')
make_level3_association(all_lw_cal_files, all_lw_asn_3, 'all_subarrays_lw')

#### 2. Run calwebb_image3

Imaging data

In [ ]:
association_files_3 = [sub160_sw_asn_3]#, sub160_lw_asn_3, sub320_sw_asn_3, sub320_lw_asn_3,
                       #sub640_sw_asn_3, sub640_lw_asn_3, full_sw_asn_3, full_lw_asn_3,
                       #all_sw_asn_3, all_lw_asn_3]

for asn in association_files_3:
    run_calwebb_image3(asn)

Grism TSO data

In [ ]:
association_files_3tso = [sub400p_lw_asn_3, sub400p_sw_asn_3, sub64p_lw_asn_3,
                            sub64p_sw_asn_3, substripe256_sw_asn_3, substripe256_lw_asn_3]
for asn in association_files_3tso:
    run_calwebb_tso3(asn)

<a id='goal_2'></a>
## Goal 2: Confirm positions of subarrays on detectors are correct

For this goal, we need to confirm that the pixels contained in the subarray files are those expected from the detector. For example, we need to be sure that the B1_SUB640 subarray is composed of pixels x = (1 - 640), y = (1408 - 2048).

There are two methods that can be used for this confirmation:

1) [Manual examination of images](#manual_examination). In this case the expected pixels are extracted from a raw full frame observation. The bias structure/bad pixels in these extracted pixels are then compared by eye to that in the subarray observation.

2) [Compare source locations](#source_locations) in the full frame vs subarray observations. Here, using calibrated or uncalibrated slope images, we use phototils to locate sources in the full frame data. The expected locations of these sources in the subarray are then calculated. These calculated positions are then compared to the measured locations of the sources in the subarray image.

Note that CAR-19 does not acquire full frame observations of the A module, so we cannot confirm the subarray location of the stripe subarray in Observation 7.

In [ ]:
goal2_output_dir = os.path.join(output_products_dir, 'subarray_position')

<a id='manual_examination'></a>
### Manual examination

In [ ]:
# B1
full = os.path.join(base_dir, 'Mirage_Output/jw01068004001_01101_00001_nrcb1_uncal.fits')
subs = [os.path.join(base_dir, 'Mirage_Output/jw01068001001_01101_00001_nrcb1_uncal.fits'),
        os.path.join(base_dir, 'Mirage_Output/jw01068002001_01101_00001_nrcb1_uncal.fits'),
        os.path.join(base_dir, 'Mirage_Output/jw01068003001_01101_00001_nrcb1_uncal.fits'),
       ]
for sub in subs:
    subarray, cropped, loc_info, comp_file = examination.compare(full_frame_file=full, subarray_file=sub,
                                                                 output_dir=output_products_dir)
    print('Comparison file saved to: {}'.format(comp_file))
    
    crop_mean, crop_med, crop_dev = sigma_clipped_stats(cropped, sigma=2, maxiters=5, cenfunc='median')
    cmin = crop_med - 3*crop_dev
    cmax = crop_med + 3*crop_dev
    
    sub_mean, sub_med, sub_dev = sigma_clipped_stats(subarray, sigma=2, maxiters=5, cenfunc='median')
    smin = crop_med - 3*crop_dev
    smax = crop_med + 3*crop_dev
    
    f, a = plt.subplots(1, 2, figsize = (15, 8))
    a[0].imshow(cropped, origin='lower', vmin=cmin, vmax=cmax)
    a[1].imshow(subarray, origin='lower', vmin=smin, vmax=smax)
    a[0].set_title('Cropped from full frame')
    a[1].set_title('Subarray')
    plt.show()

In [ ]:
# B5
full = os.path.join(base_dir, 'Mirage_Output/jw01068004001_01101_00001_nrcb5_uncal.fits')
subs = [os.path.join(base_dir, 'Mirage_Output/jw01068001001_01101_00001_nrcb5_uncal.fits'),
        os.path.join(base_dir, 'Mirage_Output/jw01068002001_01101_00001_nrcb5_uncal.fits'),
        os.path.join(base_dir, 'Mirage_Output/jw01068003001_01101_00001_nrcb5_uncal.fits'),
       ]
for sub in subs:
    subarray, cropped, loc_info, comp_file = examination.compare(full_frame_file=full, subarray_file=sub,
                                                                 output_dir=output_products_dir)
    print('Comparison file saved to: {}'.format(comp_file))
    
    crop_mean, crop_med, crop_dev = sigma_clipped_stats(cropped, sigma=2, maxiters=5, cenfunc='median')
    cmin = crop_med - 3*crop_dev
    cmax = crop_med + 3*crop_dev
    
    sub_mean, sub_med, sub_dev = sigma_clipped_stats(subarray, sigma=2, maxiters=5, cenfunc='median')
    smin = crop_med - 3*crop_dev
    smax = crop_med + 3*crop_dev
    
    f, a = plt.subplots(1, 2, figsize = (15, 8))
    a[0].imshow(cropped, origin='lower', vmin=cmin, vmax=cmax)
    a[1].imshow(subarray, origin='lower', vmin=smin, vmax=smax)
    a[0].set_title('Cropped from full frame')
    a[1].set_title('Subarray')
    plt.show()

In [ ]:
# B1 - point source subarrays
full = os.path.join(base_dir, 'Mirage_Output/jw01068004001_01101_00001_nrcb1_uncal.fits')
subs = [os.path.join(base_dir, 'Mirage_Output/jw01068005001_01101_00001_nrcb1_uncal.fits'),
        os.path.join(base_dir, 'Mirage_Output/jw01068006001_01101_00001_nrcb1_uncal.fits'),
       ]
for sub in subs:
    subarray, cropped, loc_info, comp_file = examination.compare(full_frame_file=full, subarray_file=sub,
                                                                 output_dir=output_products_dir)
    print('Comparison file saved to: {}'.format(comp_file))
    
    crop_mean, crop_med, crop_dev = sigma_clipped_stats(cropped, sigma=2, maxiters=5, cenfunc='median')
    cmin = crop_med - 3*crop_dev
    cmax = crop_med + 3*crop_dev
    
    sub_mean, sub_med, sub_dev = sigma_clipped_stats(subarray, sigma=2, maxiters=5, cenfunc='median')
    smin = crop_med - 3*crop_dev
    smax = crop_med + 3*crop_dev
    
    f, a = plt.subplots(1, 2, figsize = (15, 8))
    a[0].imshow(cropped, origin='lower', vmin=cmin, vmax=cmax)
    a[1].imshow(subarray, origin='lower', vmin=smin, vmax=smax)
    a[0].set_title('Cropped from full frame')
    a[1].set_title('Subarray')
    plt.show()

In [ ]:
# B5 - point source subarrays
full = os.path.join(base_dir, 'Mirage_Output/jw01068004001_01101_00001_nrcb5_uncal.fits')
subs = [os.path.join(base_dir, 'Mirage_Output/jw01068005001_01101_00002_nrcb5_uncal.fits'),
        os.path.join(base_dir, 'Mirage_Output/jw01068006001_01101_00002_nrcb5_uncal.fits'),
       ]
for sub in subs:
    subarray, cropped, loc_info, comp_file = examination.compare(full_frame_file=full, subarray_file=sub,
                                                                 output_dir=output_products_dir)
    print('Comparison file saved to: {}'.format(comp_file))
    
    crop_mean, crop_med, crop_dev = sigma_clipped_stats(cropped, sigma=2, maxiters=5, cenfunc='median')
    cmin = crop_med - 3*crop_dev
    cmax = crop_med + 3*crop_dev
    
    sub_mean, sub_med, sub_dev = sigma_clipped_stats(subarray, sigma=2, maxiters=5, cenfunc='median')
    smin = crop_med - 3*crop_dev
    smax = crop_med + 3*crop_dev
    
    f, a = plt.subplots(1, 2, figsize = (15, 8))
    a[0].imshow(cropped, origin='lower', vmin=cmin, vmax=cmax)
    a[1].imshow(subarray, origin='lower', vmin=smin, vmax=smax)
    a[0].set_title('Cropped from full frame')
    a[1].set_title('Subarray')
    plt.show()

<a id='source_locations'></a>
### Compare source locations

In [ ]:
# B1 - SUB160, SUB320, SUB640, FULL
full = os.path.join(base_dir, 'Pipeline_Level2/jw01068004001_01101_00002_nrcb1_cal.fits')
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb1_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb1_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb1_cal.fits']
subs = [os.path.join(base_dir, s) for s in subs]
locations.run(full, subs, output_dir=output_products_dir)

In [ ]:
# B2 - SUB160, SUB320, SUB640, FULL
full = os.path.join(base_dir, 'Pipeline_Level2/jw01068004001_01101_00003_nrcb2_cal.fits')
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb2_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb2_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb2_cal.fits']

#subs = ['Pipeline_Level2/jw01068002001_01101_00001_nrcb2_cal.fits',
#        'Pipeline_Level2/jw01068003001_01101_00001_nrcb2_cal.fits']
subs = [os.path.join(base_dir, s) for s in subs]
locations.run(full, subs)

In [ ]:
# B3 - SUB160, SUB320, SUB640, FULL
full = os.path.join(base_dir, 'Pipeline_Level2/jw01068004001_01101_00002_nrcb3_cal.fits')
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb3_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb3_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb3_cal.fits']
subs = [os.path.join(base_dir, s) for s in subs]
locations.run(full, subs)

In [ ]:
# B4 - SUB160, SUB320, SUB640, FULL
full = os.path.join(base_dir, 'Pipeline_Level2/jw01068004001_01101_00002_nrcb4_cal.fits')
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb4_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb4_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb4_cal.fits']
subs = [os.path.join(base_dir, s) for s in subs]
locations.run(full, subs)

In [ ]:
# B5 - SUB160, SUB320, SUB640, FULL
full = os.path.join(base_dir, 'Pipeline_Level2/jw01068004001_01101_00001_nrcb5_cal.fits')
subs = ['Pipeline_Level2/jw01068001001_01101_00001_nrcb5_cal.fits',
        'Pipeline_Level2/jw01068002001_01101_00001_nrcb5_cal.fits',
        'Pipeline_Level2/jw01068003001_01101_00001_nrcb5_cal.fits']
subs = [os.path.join(base_dir, s) for s in subs]
locations.run(full, subs)

In [ ]:
# B1 - SUB64P, SUB600P
full = os.path.join(base_dir, 'Pipeline_Level2/jw01068004001_01101_00002_nrcb1_cal.fits')
subs = ['Pipeline_Level2/jw01068006001_01101_00001_nrcb1_cal.fits',
        'Pipeline_Level2/jw01068005001_01101_00001_nrcb1_cal.fits']
subs = [os.path.join(base_dir, s) for s in subs]
locations.run(full, subs)

In [ ]:
# B5 - SUB64P, SUB600P
full = os.path.join(base_dir, 'Pipeline_Level2/jw01068004001_01101_00001_nrcb5_cal.fits')
subs = ['Pipeline_Level2/jw01068006001_01101_00002_nrcb5_cal.fits',
        'Pipeline_Level2/jw01068005001_01101_00002_nrcb5_cal.fits']
subs = [os.path.join(base_dir, s) for s in subs]
locations.run(full, subs)

<a id='goal_3'></a>
## Goal 3: Confirm telescope pointing places the source at the proper location in aperture

In this goal, we attempt to confirm that the telescope pointing for each subarray is correct. There are three methods that can be used to do this:

1) [Compare source locations to 2MASS catalog](#twomass_comp). Using photutils, locate sources in the input calibrated slope image. Then, starting from a 2MASS catalog, calculate the expected (x, y) locations of sources in the observation. Compare the measured and calculated source positions.

2) [Compare target location to expected location](#targ_star_comp). For the point source subarray observations (Observations 5 and 6 in the APT file), the APT target is an actual star with a known RA, Dec. Calculate the expected (x, y) location of this RA, Dec, and compare to the measured star location. This is essentially the same check as is done in the test described above.

3) [Check that target RA, Dec in file matches position in APT](#targ_ra_dec_comp). This test simply confirms that the target RA, Dec from the APT file is present in the TARG_RA and TARG_DEC header keywords of the observation files.

In [ ]:
#filenames = ['Pipeline_Level2/jw01068005001_01101_00001_nrcb1_cal.fits',
#             'Pipeline_Level2/jw01068005001_01101_00002_nrcb5_cal.fits',
#             'Pipeline_Level2/jw01068006001_01101_00001_nrcb1_cal.fits',
#             'Pipeline_Level2/jw01068006001_01101_00002_nrcb5_cal.fits']
#filenames = [os.path.join(base_dir, s) for s in filenames]

filenames = sorted(glob(os.path.join(base_dir, 'Pipeline_Level2/jw01068*cal.fits')))

In [ ]:
filenames

<a id='twomass_comp'></a>
### Compare source locations against expected locations from independent 2MASS catalog

In [ ]:
for filename in filenames:
    med_offset, dev_offset, mean_unc, offsets = \
              pointing.check_pointing_using_2mass_catalog(filename, out_dir=output_products_dir)

<a id='targ_star_comp'></a>
### Compare target star location against expected location given its (RA, Dec)

Note that the target star is only used for the point source subarrays. This check should be redundant after running check_pointing_using_2mass_catalog().

In [ ]:
ptsrc_filenames = ['Pipeline_Level2/jw01068005001_01101_00001_nrcb1_cal.fits',
                   'Pipeline_Level2/jw01068005001_01101_00002_nrcb5_cal.fits',
                   'Pipeline_Level2/jw01068006001_01101_00001_nrcb1_cal.fits',
                   'Pipeline_Level2/jw01068006001_01101_00002_nrcb5_cal.fits',
                   'Pipeline_Level2/jw01068007001_01101_00001-seg001_nrca1_cal.fits',
                   'Pipeline_Level2/jw01068007001_01101_00002-seg001_nrca5_cal.fits']
ptsrc_filenames = [os.path.join(base_dir, s) for s in ptsrc_filenames]

In [ ]:
for filename in ptsrc_filenames:
    diff = pointing.check_pointing_target_star(filename, out_dir=output_products_dir)

In [ ]:
Still need SUBTRIPE256

<a id='targ_ra_dec_comp'></a>
### Check that the TARG_RA, TARG_DEC in the file matches that from the APT file

In [ ]:
#lw_files = ['Pipeline_Level2/jw01068005001_01101_00002_nrcb5_cal.fits']
#lw_files = [os.path.join(base_dir, lfile) for lfile in lw_files]

files = glob(os.path.join(base_dir, 'Pipeline_Level2/jw01068*_cal.fits')

In [ ]:
for file in files:
    header0 = fits.getheader(file)
    obs = int(header0['OBSERVTN'])
    if obs <= 4:
        expected_ra = point_ra
        expected_dec = point_dec
    else:
        expected_ra = extended_ra
        expected_dec = extended_dec
    header_ra, header_dec = pointing.check_targ_ra_dec(header0, expected_ra, expected_dec)

<a id='goal_4'></a>
## Goal 4: Charge Accumulation Rate

Show that the charge accumulation rate in the subarray files is consistent with that in the full frame data. There are three methods that can be used for this:

1) [Compare photometry in subarrays vs full frame](#photometry_comparison). Using photutils on calibrated or uncalibrated slope images, locate sources and then do simple aperture photometry on the subarray as well as full frame data. After matching catalogs (and optionally filtering out sources containing any bad pixels), compare the measured signal rates of the sources. 

2) [Compare level 3 source catalogs](#compare_level3_catalog). Read in the level 3 pipeline-output source catalogs for a product made from full frame observations, as well as a product from subarray observations. Perform catalog matching and compare the photometry results.

3) [Examine ratio images with the same pointing](#ratios). In cases where we have a subarray observation at the same pointing as a full frame observation (i.e. the same RA, Dec at the reference location, and the same roll angle), create a ratio image of the calibrated or uncalibrated slope images. Examine the ratio image, and check its median value.

In [ ]:
countrate_output_dir = os.path.join(output_products_dir, 'count_rate_comparison')

<a id='photometry_comparison'></a>
### Compare photometry results of sources in subarrays and full frame data

First, check LW data using calibrated slope images

In [ ]:
fullframe_file = os.path.join(base_dir, 'Pipeline_level2/jw01068004001_01101_00001_nrcb5_cal.fits')

In [ ]:
subfiles = sorted(glob(os.path.join(base_dir, 'Pipeline_level2/jw01068001*nrcb5_cal.fits')))

for f in subfiles:
    print(f)

In [ ]:
# sub_dq and full_dq columns: if True, then there is at least one non-zero value
# in the DQ array within the photometry aperture in the subarray or full frame
# images, respectively. In some cases this may not matter. But in some cases the
# flagged pixel defintely has an effect.

# d_phot_p is the percentage difference in the photometry between the subarray
# and full frame cases. Eqn is (full frame - subarray) / full frame, so negative
# means the source is brighter in the subarray data
for subarray_file in subfiles[0:1]:
    print('Trying: {}'.format(subarray_file))
    phot_tab, clean_phot_tab = count_rates.compare_rate_images(subarray_file, fullframe_file,
                                                               output_products_dir)

Look at the resulting photometry table

In [ ]:
phot_tab

Look at the table of "clean" sources. This table excludes sources where there is a bad pixel flagged in the subarray or full frame DQ arrays at the locations of the sources

In [ ]:
clean_phot_tab

Now look at SW data calibrated slope images

In [ ]:
fullframe_file = os.path.join(base_dir, 'Pipeline_level2/jw01068004001_01101_00001_nrcb1_cal.fits')

In [ ]:
subfiles = sorted(glob(os.path.join(base_dir, 'Pipeline_level2/jw01068002*nrcb1_cal.fits')))
for subfile in subfiles:
    print(subfile)

In [ ]:
for subarray_file in subfiles:
    print('Trying: {}'.format(subarray_file))
    phot_tab, clean_phot_tab = count_rates.compare_rate_images(subarray_file, fullframe_file,
                                                               out_dir=output_products_dir)

<a id='compare_level3_catalog'></a>
### Compare data in the level 3 output source catalogs

In [ ]:
def generate_comparison_filename(cat1, cat2):
    """Construct a filename which will be used to save the table containing
    comparitive photometry from two source catalogs
    """
    cat1base = os.path.basename(cat1).replace('.ecsv', '')
    cat2base = os.path.basename(cat2).replace('.ecsv', '')
    comp_name = 'photometry_comparison_{}_{}.txt'.format(cat1base, cat2base)
    return comp_name

In [ ]:
catalog1 = os.path.join(base_dir, 'Pipeline_level3/sub160_lw_cat.ecsv')
catalog2 = os.path.join(base_dir, 'Pipeline_level3/full_lw_cat.ecsv')
comparison_output_table = generate_comparison_filename(catalog1, catalog2)

In [ ]:
comparison_output_table

In [ ]:
comp_tab = count_rates.compare_level3_catalogs(catalog1, catalog2, comparison_output_table,
                                               out_dir=output_products_dir)

In [ ]:
comp_tab

In [ ]:
comp_tab['delta_flux_perc'].data

In [ ]:
catalog1 = os.path.join(base_dir, 'Pipeline_level3/sub160_sw_cat.ecsv')
catalog2 = os.path.join(base_dir, 'Pipeline_level3/full_sw_cat.ecsv')
comparison_output_table = generate_comparison_filename(catalog1, catalog2)

In [ ]:
comp_tab = count_rates.compare_level3_catalogs(catalog1, catalog2, comparison_output_table,
                                               out_dir=output_products_dir)

In [ ]:
comp_tab

In [ ]:
comp_tab['delta_flux_perc'].data

<a id='ratios'></a>
### Examine ratio images of images with the same pointing

In [ ]:
# B1 through B5 - grab only the first exposure
sub160_pointing1 = sorted(glob(os.path.join(base_dir, 'Mirage_Output/jw01068001001_01101_00001*uncal.fits')))
sub320_pointing1 = sorted(glob(os.path.join(base_dir, 'Mirage_Output/jw01068002001_01101_00001*uncal.fits')))
sub640_pointing1 = sorted(glob(os.path.join(base_dir, 'Mirage_Output/jw01068003001_01101_00001*uncal.fits')))
full_pointing1 = sorted(glob(os.path.join(base_dir, 'Mirage_Output/jw01068004001_01101_00001*uncal.fits')))
full_b1_b5_only = [full_pointing1[0], full_pointing1[-1]]

# B1 and B5 only - grab only the second exposure (the first exposure is the TA)
sub400p_pointing2 = sorted(glob(os.path.join(base_dir, 'Mirage_Output/jw01068005001_01101_00002*uncal.fits')))
sub64p_pointing2 = sorted(glob(os.path.join(base_dir, 'Mirage_Output/jw01068006001_01101_00002*uncal.fits')))

# I don't think we can use the TSO observation here because of the small shift in y that is
# used to get the target to the reference location
#sub256_pointing1 = sorted(glob(os.path.join(base_dir, 'Mirage_Output/jw01068007001_01101_00002*uncal.fits')))

In [ ]:
def calculate_ratios(file1, file2):
    with fits.open(file1) as f:
        detector = f[0].header['DETECTOR']
        aperture1 = f[0].header['SUBARRAY']
    with fits.open(file2) as f:
        aperture2 = f[0].header['SUBARRAY']

    ratio_image, ratio_med = count_rates.ratio_images(file1, file2, out_dir=countrate_output_dir)
    print('Median of ratio image: {}'.format(ratio_med))
    f, a = plt.subplots(figsize=(10,10))
    a.imshow(ratio_image, origin='lower', vmin=0.2, vmax=20)
    a.set_title('{} / {}, {}'.format(aperture1, aperture2, detector))
    plt.show()

In [ ]:
# This should be looping over detectors
for file_160, file_full in zip(sub160_pointing1, full_pointing1):
    calculate_ratios(file_160, file_full)

In [ ]:
# This should be looping over detectors
for file_320, file_full in zip(sub320_pointing1, full_pointing1):
    calculate_ratios(file_320, file_full)

In [ ]:
# This should be looping over detectors.
for file_640, file_full in zip(sub640_pointing1, full_pointing1):
    calculate_ratios(file_640, file_full)

In [ ]:
# This should be looping over detectors. In this case we want only the B1
# and B5 files from the full frame observation
for file_400p, file_full in zip(sub400p_pointing2, full_pointing1):
    calculate_ratios(file_400p, file_full)

In [ ]:
# This should be looping over detectors. In this case we want only the B1
# and B5 files from the full frame observation
for file_64p, file_full in zip(sub64p_pointing2, full_pointing1):
    calculate_ratios(file_64p, file_full)

<a id='goal_5'></a>
## Goal 5: Identify and characterize latency

Using _cal.fits files:

0. Mario has HST/WFC3 scripts that are a good place to start
1. Locate and do aperture photometry on sources in exposure 1
2. Locate and do aperture photometry on sources in exposure 2
3. In exposure 2, do aperture photometry on locations from exposure 1, excluding locations where there is also a source in exposure 2.
4. Create plot of "empty" location photometry vs time since dither?
5. Repeat for subsequent exposures

In [ ]:
files = sorted(glob('Pipeline_Level1/jw01068001001*nrcb5_rateints.fits'))
files = [os.path.join(base_dir, f) for f in files]

In [ ]:
latency.check(files)